# Face Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensein/senselab/blob/main/tutorials/video/face_analysis.ipynb)

This tutorial demonstrates how to use Senselab's Face Analysis API for analyzing human faces. At the moment, this API supports available analysis through [DeepFace](https://github.com/serengil/deepface).

## Setup

Let's get started by installing Senselab and importing the necessary modules from Senselab for processing images/videos and performing face analysis.

In [ ]:
%pip install senselab['video']

In [ ]:
from senselab.video.tasks.face_analysis import (
    analyze_face_attributes,
    extract_face_embeddings,
    recognize_faces,
    verify_faces
)

from senselab.video.data_structures.video import Video

import pprint
from IPython.display import Image, display, Video as IPyVideo

In [ ]:
!mkdir -p tutorial_images
!mkdir -p tutorial_images/db
!wget -O tutorial_images/sally_1.jpg https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/sally_1.jpg
!wget -O tutorial_images/db/group.jpg https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/db/group.jpg
!wget -O tutorial_images/db/sally_2.jpg https://raw.githubusercontent.com/sensein/senselab/main/src/tests/data_for_testing/face_data/db/sally_2.jpg

In [ ]:
TUTORIAL_PATH = "tutorial_images"
DB_PATH = TUTORIAL_PATH + "/db"

SALLY_1 = TUTORIAL_PATH + "/sally_1.jpg"
SALLY_2 = DB_PATH + "/sally_2.jpg"
GROUP = DB_PATH + "/group.jpg"

SALLY_VID_PATH = TUTORIAL_PATH + "/sally_vid.mp4"
SALLY_VID = Video.from_filepath(str(SALLY_VID_PATH))

## Preview Inputs

Let’s display the input media we’ll use for the tutorial:

In [ ]:
# Show images
print("SALLY_1 (Query Image)")
display(Image(SALLY_1, width=300))

print("SALLY_2 (Database Image)")
display(Image(SALLY_2, width=300))

print("GROUP (Database Image)")
display(Image(GROUP, width=300))

print("SALLY_VID (Query Video)")
display(IPyVideo(SALLY_VID_PATH, embed=True, width=500))

## Recognize Faces

### In an image
Let's begin by checking if individuals in a particular image can be matched with any indivduals in a database of images (in this context a folder containing different images).

In [ ]:
recognized = recognize_faces(SALLY_1, DB_PATH)

# Since this is a single image, we expect recognize to be a list of length 1
print(len(recognized))

# Since there is only one individual in the image, we expect there to be only one DetectedFace
source = None
if len(recognized) > 0:
    source = recognized[0]
    print(len(recognized[0]))

# With that we can look into the details to understand the matches to images in the database
if source and len(source) > 0:
    source_face = source[0]
    for matched_face in source_face.face_match:
        pprint.pprint(matched_face)

### In a video
Next, let's compare faces in a video by frame with any indivdual in a database of images (in this context a folder containing different images).

In [ ]:
recognized = recognize_faces(SALLY_VID, DB_PATH)

# Since this is a video, we expect recognize to be a list equal to the number of frames in the video
print(len(recognized) == len(SALLY_VID.frames))

# Iterate through the frames and identify matches
for frame_ix, frame in enumerate(recognized):
  print(f"Frame {frame_ix + 1}")
  for face_ix, face in enumerate(frame):
    print(f"Face {face_ix + 1}")
    pprint.pprint(face)

# Verify Faces

Here, we can compare 2 images to confirm whether the same person is in both. 

Note: This currently only works with images containing a single individual. If working with multi-individual images or videos, use the `recognize_faces` function.

In [ ]:
verify = verify_faces(SALLY_1, SALLY_2)

# We expect verified to be true
pprint.pprint(verify)

## Extract Embeddings

This function returns a numerical representation of the face that can be used for similarity comparison, clustering, or custom analysis.

In [ ]:
embeddings = extract_face_embeddings(GROUP)

# Since this is a single image, we expect a list of one frame
print(len(embeddings) == 1)

# Inspect embeddings for each face
if embeddings and embeddings[0]:
    print("Printing first 10 values of each face embedding")
    for face_ix, face in enumerate(embeddings[0]):
        print(f"Face {face_ix + 1} Embedding {face.embedding[:10]}")

### Embeddings from video

You can also extract embeddings frame-by-frame from a video:

In [ ]:
video_embeddings = extract_face_embeddings(SALLY_VID)

# Print the number of frames and number of detected faces per frame
print(f"Video has {len(video_embeddings)} frames")
for frame_ix, frame in enumerate(video_embeddings):
    print(f"Frame {frame_ix + 1} - Faces detected: {len(frame)}")
    for face_ix, face in enumerate(frame):
        print(f"  Face {face_ix + 1} Embedding (first 5 values): {face.embedding[:5]}")

## Analyze Face Attributes

This function analyzes and returns facial attributes such as age, emotion, gender, and race.

In [ ]:
attributes = analyze_face_attributes(SALLY_1, actions=["age", "gender", "emotion", "race"], detector_backend="retinaface")

# Since this is a single image, we expect a list of one frame
print(len(attributes) == 1)

# Inspect attributes for each detected face
if attributes and attributes[0]:
    for ix, face in enumerate(attributes[0]):
        print(f"Face {ix + 1}")
        pprint.pprint(face.attributes)

### Attributes from video

Attributes can also be extracted from videos:

In [ ]:
video_attributes = analyze_face_attributes(SALLY_VID, actions=["age", "gender", "emotion", "race"])

# Print attributes per frame
for frame_ix, frame in enumerate(video_attributes):
    print(f"Frame {frame_ix + 1}")
    for face_ix, face in enumerate(frame):
        print(f"  Face {face_ix + 1}")
        pprint.pprint(face.attributes)

In this tutorial, we explored how to use Senselab’s face analysis API for:

- Recognizing individuals in images or videos
- Verifying whether two images represent the same person
- Extracting numerical embeddings of faces
- Analyzing age, gender, race, and emotion

See the [Senselab Documentation](https://sensein.group/senselab/senselab.html) for function-specific parameters such as recognition model, alignment settings, threshold, detctor backend, and more.